# Variational Classifier

For this task, we will implement two quantum variational classifiers. First, we will train a quantum variational circuit to emulate the parity function 
$$
f : x \in \{0,1\}^{\otimes n} \rightarrow y = \begin{cases} 
1 & \text{if odd number of 1's in } x \\
0 & \text{else}.
\end{cases}
$$
And second, we will train another quantum variational circuit to recognize the first two classes of flowers in the iris dataset.

## 1. Parity Classification

We follow the basic workflow in Quantum Machine Learning: 

1. **Encoding**: we encode the model's input into the initial state of the quantum circuit. In the present example, basis encoding lends itself most naturally, since the model's input is a bit string.

2. **Evolving**: we evolve the input state by applying a series of parameterized unitary operators, with the parameters acting as the weights of our quantum neural network.

3. **Measurement**: we perform a specific measurement on the final state of our circuit. The resulting value will be our model's prediction of the function's output for the specified encoded input in step 1.

4. **Optimizing**: we use an optimizer to update our parameters used in step 2 using a gradient-based method. In our present case, we use `NesterovMomentumOptimizer`, which implements the Nesterov Momentum optimization technique that improves the convergence speed of gradient descent algorithms by incorporating momentum.

In step 2, we evolve the state by applying *layers*, each composed of an arbitrary rotation (most general unitary with 3 free parameters) on each individual qubit, along with a ring of CNOT gates. Our full variational circuit for two layers should look like this:

<div style="text-align:center">
    <img src="img/parity_function_image.png" alt="Alt Text" style="width:900px;"/>
</div>

where the block "$\ket\psi$" denotes our basis encoding routine.

We now begin implementing the model by importing the necessary packages and creating our quantum device to implement the circuit. We choose the default pennylane simulator.

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer 
import matplotlib.pyplot as plt

In [2]:
dev = qml.device("default.qubit") #default pennylane simulator

We then define a `state_prepararion` function that simply does the basis encoding for us. We define a `layer` function that takes the $4 \times 3$ array corresponding to the layer's weights (3 wights per each of the 4 qubits) and implements the corresponding rotation as well as the ring of CNOTS. 

We next define our variational circuit that performs all of the above + performs a measurment on our system. And finally we define the `variatoinal_classifier` function that adds to our output a bias. This will be the model's final output to be referenced later by our `cost` function.

In [3]:
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

def layer(layer_weights):
    for wire in range(4):
        #unpacking the layer's weights and passing them as angles to qml.Rot()
        qml.Rot(*layer_weights[wire], wires=wire) 

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)


@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x) #Step 1: (Basis) Encoding

    for layer_weights in weights:
        layer(layer_weights) #Step 2: Evolving

    return qml.expval(qml.PauliZ(0)) #Step 3: Measurement


#incorporating a bias into the output
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


Our optimizer, ofcourse, neeeds a cost function to minimize. We take the standard mean square difference as our loss function, which gives the "distance" between each input's label and our model's output.

We also define an "accuracy" fucntion which gives a measure of the accuracy of the model's output, for monitoring purposes.

In [4]:
def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

We now load in the training data into $\verb|X|$, the corresponding labels into $\verb|Y|$, and shift the labels from {0, 1} to {-1, 1}, which will make it wasier for our model to learn.

In [5]:
data = np.loadtxt("data/parity_train.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")

x = [0 0 0 1], y = 1
x = [0 0 1 0], y = 1
x = [0 1 0 0], y = 1
x = [0 1 0 1], y = -1
x = [0 1 1 0], y = -1
x = [0 1 1 1], y = 1
x = [1 0 0 0], y = 1
x = [1 0 0 1], y = -1
x = [1 0 1 1], y = 1
x = [1 1 1 1], y = -1


We now initialize our model. We need 4 qubits since the model's input is a 4-bit string. We specify the number of layers to 2. And we initialize the weights pseudorandomly (with a specified seed for reproducability).

In [6]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights: \n", weights_init)
print("Bias: ", bias_init)

Weights: 
 [[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]
Bias:  0.0


We then initialize our optimizer with a learning rate of 0.5, and specify the learning batch size to 5 inputs.

In [7]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

We now begin the learning process. We iterate over the following steps:
1. we randomly sample 5 inputs from our input array $\verb|X|$, along with their labels.
2. we pass the sampled data along with the current iteration's weights to the optimizer
3. the optimizer updates the weights (and bias) to new "better" values such that the model better approximates $f(x)$
4. given the new weights, we run the model over all of our inputs and get the predictions for the current iteration
5. we print out the current cost and accuracy of the model

In [8]:
weights = weights_init
bias = bias_init

#Step 4: Optimizing
for it in range(60):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 2.3147651 | Accuracy: 0.5000000
Iter:    2 | Cost: 1.9664866 | Accuracy: 0.5000000
Iter:    3 | Cost: 1.9208589 | Accuracy: 0.5000000
Iter:    4 | Cost: 2.6276126 | Accuracy: 0.5000000
Iter:    5 | Cost: 0.9323119 | Accuracy: 0.6000000
Iter:    6 | Cost: 1.1903549 | Accuracy: 0.5000000
Iter:    7 | Cost: 2.0508989 | Accuracy: 0.4000000
Iter:    8 | Cost: 1.1275531 | Accuracy: 0.6000000
Iter:    9 | Cost: 1.1659803 | Accuracy: 0.6000000
Iter:   10 | Cost: 1.1349618 | Accuracy: 0.6000000
Iter:   11 | Cost: 0.9994063 | Accuracy: 0.6000000
Iter:   12 | Cost: 1.0812559 | Accuracy: 0.6000000
Iter:   13 | Cost: 1.2863155 | Accuracy: 0.6000000
Iter:   14 | Cost: 2.2658259 | Accuracy: 0.4000000
Iter:   15 | Cost: 1.1323724 | Accuracy: 0.6000000
Iter:   16 | Cost: 1.3439737 | Accuracy: 0.8000000
Iter:   17 | Cost: 2.0076168 | Accuracy: 0.6000000
Iter:   18 | Cost: 1.2685760 | Accuracy: 0.5000000
Iter:   19 | Cost: 1.6762475 | Accuracy: 0.5000000
Iter:   20 | Cost: 1.1868237 | 

After the learning process is completed, the model now computes $f(x)$ to a very high accuracy for all of the training data. We now wish to test how well it generalizes by feeding it new "unseen" data.

In [9]:
data = np.loadtxt("data/parity_test.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, prediction = {p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

x = [0 0 0 0], y = -1, prediction = -1.0
x = [0 0 1 1], y = -1, prediction = -1.0
x = [1 0 1 0], y = -1, prediction = -1.0
x = [1 1 1 0], y = 1, prediction = 1.0
x = [1 1 0 0], y = -1, prediction = -1.0
x = [1 1 0 1], y = 1, prediction = 1.0
Accuracy on unseen data: 1.0


Our job is now done since the model continues to perform well.

our next example in "Iris_Classification.ipynb" shows an extention of the quantum variational classifier model to a deal with a problem with a slightly more complex data set.

### References

Variational Classifier [Pennylane Demo](https://pennylane.ai/qml/demos/tutorial_variational_classifier/) 